In [ ]:
import tensorflow as tf
print('TensorFlow Version %s' % tf.__version__)

In [ ]:
import sys
print('Python Version %s' % sys.version)

In [ ]:
!pip list

In [ ]:
%%bash
conda list

In [ ]:
%%html

<iframe src="https://datascienceonaws.com" width="800px" height="600px"/>

# 1. Install the CLI tools

_Note:  Make sure the SageMaker role has access to create EKS Cluster._

In [5]:
%%bash

source ~/.bash_profile

#### Install `eksctl`
# To get started we'll first install the `awscli` and `eksctl` CLI tools. [eksctl](https://eksctl.io) simplifies the process of creating EKS clusters.

pip install awscli --upgrade --user

curl --silent --location "https://github.com/weaveworks/eksctl/releases/download/latest_release/eksctl_$(uname -s)_amd64.tar.gz" | tar xz -C /tmp

sudo mv /tmp/eksctl /usr/local/bin

eksctl version

#### Install `kubectl`
# `kubectl` is a command line interface for running commands against Kubernetes clusters. 
# Run the following to install Kubectl

curl -o kubectl https://amazon-eks.s3-us-west-2.amazonaws.com/1.14.6/2019-08-22/bin/linux/amd64/kubectl

chmod +x ./kubectl

sudo mv ./kubectl /usr/local/bin

kubectl version --short --client

#### Install `aws-iam-authenticator`

curl -o aws-iam-authenticator https://amazon-eks.s3-us-west-2.amazonaws.com/1.14.6/2019-08-22/bin/linux/amd64/aws-iam-authenticator

chmod +x ./aws-iam-authenticator

sudo mv aws-iam-authenticator /usr/local/bin

aws-iam-authenticator version

#### Install jq and envsubst (from GNU gettext utilities) 
sudo yum -y install jq gettext

#### Verify the binaries are in the path and executable
for command in kubectl jq envsubst
  do
    which $command &>/dev/null && echo "$command in path" || echo "$command NOT FOUND"
  done

echo "Completed"

[ℹ]  version.Info{BuiltAt:"", GitCommit:"", GitTag:"0.13.0"}
Client Version: v1.14.7-eks-1861c5
{"Version":"v0.4.0","Commit":"c141eda34ad1b6b4d71056810951801348f8c367"}
Loaded plugins: dkms-build-requires, priorities, update-motd, upgrade-helper,
              : versionlock
Package jq-1.5-1.2.amzn1.x86_64 already installed and latest version
Package gettext-0.18.1.1-9.1.11.amzn1.x86_64 already installed and latest version
Nothing to do
kubectl in path
jq in path
envsubst in path
Completed


ERROR: boto3 1.11.14 has requirement botocore<1.15.0,>=1.14.14, but you'll have botocore 1.15.11 which is incompatible.
You should consider upgrading via the 'pip install --upgrade pip' command.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 41.0M  100 41.0M    0     0  15.9M      0  0:00:02  0:00:02 --:--:-- 15.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17.7M  100 17.7M    0     0  12.1M      0  0:00:01  0:00:01 --:--:-- 12.1M


# 2. Configure the Environment Variables

In [9]:
%%bash

source ~/.bash_profile

export AWS_REGION=us-west-2
echo "export AWS_REGION=${AWS_REGION}" | tee -a ~/.bash_profile

export AWS_CLUSTER_NAME=cluster
echo "export AWS_CLUSTER_NAME=${AWS_CLUSTER_NAME}" | tee -a ~/.bash_profile

echo "Completed"

export AWS_REGION=us-west-2
export AWS_CLUSTER_NAME=cluster
Completed


# 3. Create the EKS Cluster

_Note:  Creating a cluster may take about 15 mins._

In [10]:
%%bash

source ~/.bash_profile

eksctl create cluster \
    --name ${AWS_CLUSTER_NAME} \
    --version 1.14 \
    --region ${AWS_REGION} \
    --nodegroup-name cpu-nodes \
    --node-type c5.xlarge \
    --nodes 5 \
    --node-volume-size 100 \
    --node-zones us-west-2a \
    --timeout=40m \
    --zones=us-west-2a,us-west-2b,us-west-2c \
    --alb-ingress-access \
    --auto-kubeconfig
    
echo "Completed"

[ℹ]  eksctl version 0.13.0
[ℹ]  using region us-west-2
[ℹ]  subnets for us-west-2a - public:192.168.0.0/19 private:192.168.96.0/19
[ℹ]  subnets for us-west-2b - public:192.168.32.0/19 private:192.168.128.0/19
[ℹ]  subnets for us-west-2c - public:192.168.64.0/19 private:192.168.160.0/19
Completed


Error: unable to find image "ami-0c13bb9cbfd007e56": UnauthorizedOperation: You are not authorized to perform this operation.
	status code: 403, request id: 32159135-6f3f-4da5-8625-39eb73053f5c


![](img/launch_eks.png)

# Please wait until the EKS cluster is succesfully deployed before you continue!

![](img/verify_eks.png)

# 4. Associate IAM Policies with EKS Worker Nodes


In [ ]:
%%bash

### Source the environment
source ~/.bash_profile

### Create more environment variables
export STACK_NAME=$(eksctl get nodegroup --cluster ${AWS_CLUSTER_NAME} -o json | jq -r '.[].StackName')
echo "export STACK_NAME=${STACK_NAME}" | tee -a ~/.bash_profile

export INSTANCE_ROLE_NAME=$(aws cloudformation describe-stacks --stack-name $STACK_NAME --output text --query "Stacks[0].Outputs[1].OutputValue" | sed -e 's/.*\///g')
echo "export INSTANCE_ROLE_NAME=${INSTANCE_ROLE_NAME}" | tee -a ~/.bash_profile

export INSTANCE_PROFILE_ARN=$(aws cloudformation describe-stacks --stack-name $STACK_NAME | jq -r '.Stacks[].Outputs[] | select(.OutputKey=="InstanceProfileARN") | .OutputValue')
echo "export INSTANCE_PROFILE_ARN=${INSTANCE_PROFILE_ARN}" | tee -a ~/.bash_profile

#### Allow Access from/to the Elastic Container Registry (ECR)
# This allows our cluster worker nodes to load custom Docker images (ie. models) from ECR.  We will load these custom Docker images in a later section.
aws iam attach-role-policy --role-name $INSTANCE_ROLE_NAME --policy-arn arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess

### Associated IAM and OIDC
# To use IAM roles for service accounts in your cluster, you must create an OIDC identity provider in the IAM console.  See https://docs.aws.amazon.com/eks/latest/userguide/enable-iam-roles-for-service-accounts.html for more info.
eksctl utils associate-iam-oidc-provider --cluster ${AWS_CLUSTER_NAME} --approve
aws eks describe-cluster --name ${AWS_CLUSTER_NAME} --region ${AWS_REGION} --query "cluster.identity.oidc.issuer" --output text

echo "Completed"

# 5. Update `~/.kube/config` with our new EKS cluster

In [15]:
%%bash

source ~/.bash_profile

aws eks --region ${AWS_REGION} update-kubeconfig --name ${AWS_CLUSTER_NAME} 


An error occurred (AccessDeniedException) when calling the DescribeCluster operation: User: arn:aws:sts::835319576252:assumed-role/AmazonSageMaker-ExecutionRole-20191006T135881/SageMaker is not authorized to perform: eks:DescribeCluster on resource: arn:aws:eks:us-west-2:835319576252:cluster/cluster


# 6. Verify Your EKS Cluster


In [16]:
%%bash

kubectl get ns

### EXPECTED OUTPUT ###
# NAME              STATUS   AGE
# default           Active   12m
# kube-node-lease   Active   13m
# kube-public       Active   13m
# kube-system       Active   13m


The connection to the server localhost:8080 was refused - did you specify the right host or port?


In [17]:
%%bash

kubectl get nodes

### EXPECTED OUTPUT ###
# NAME                                            STATUS   ROLES    AGE    VERSION
# ip-XXX-XXX-XXX-XXX.us-west-2.compute.internal   Ready    <none>   7m3s   v1.14.7-eks-1861c5
# ip-XXX-XXX-XXX-XXX.us-west-2.compute.internal   Ready    <none>   7m4s   v1.14.7-eks-1861c5

The connection to the server localhost:8080 was refused - did you specify the right host or port?
